In [ ]:
import json
import sys
from dotenv import load_dotenv, find_dotenv

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

In [ ]:
import aisuite as ai
from aisuite.utils.tool_manager import ToolManager  # Import your ToolManager class

client = ai.Client()
tool_manager = ToolManager()

In [ ]:
# Mock tool functions.
def get_current_temperature(location: str, unit: str):
    # Simulate fetching temperature from an API
    return {"location": location, "unit": unit, "temperature": 72}

tool_manager.add_tool(get_current_temperature)
messages = [{"role": "user", "content": "What is the current temperature in San Francisco in Celsius?"}]

In [ ]:
# model = "anthropic:claude-3-5-sonnet-20240620"
# model = openai:gpt-4o
# model = mistral:mistral-large-latest
model = "aws:anthropic.claude-3-haiku-20240307-v1:0"
# model = "groq:llama-3.1-70b-versatile"
response = client.chat.completions.create(
    model=model, messages=messages, tools=tool_manager.tools())

In [ ]:
from pprint import pprint
pprint(vars(response.choices[0].message))

In [ ]:
if response.choices[0].message.tool_calls:
    tool_results, result_as_message = tool_manager.execute_tool(response.choices[0].message.tool_calls)
    messages.append(response.choices[0].message) # Model's function call message
    messages.append(result_as_message[0])

    final_response = client.chat.completions.create(
        model=model, messages=messages, tools=tool_manager.tools())
    print(final_response.choices[0].message.content)

In [ ]:
# Now, test without tool calling.
messages = [{"role": "user", "content": "What is the capital of California?"}]
response = client.chat.completions.create(
    model=model, messages=messages)
print(response.choices[0].message.content)